### UC Berkeley, MICS, W202-Cryptography
### Week 03 Breakout 5

### Rabin Signatures

In RSA, we used odd exponents for e and d.

Rabin Signatures are based on using 2 as one of the exponents (squaring) and using 1/2 (square root) for the other exponent.

Since we are finding square roots (quadratic residue) in modulo n where n = pq and both p and q are prime, we have 4 square roots for each square (if the roots exist).  Essentially a square in modulo prime has 2 square roots.  So both p and q have 2 square roots each.  So n has 4 square roots.

(Note: consider the caveat "if the square roots exist" applies to all of this.  I won't keep repeating it here.)

The first advantage we notice is that both e and d will be public, along with n.  Essentially only p and q need to be kept secret.  

Squaring in modulo n is computationally easy, and it only has 1 solution, so it's good for the public part.

Finding a square root in modular n (unless we know p, q and p, q were both selected under strict constraints) is computationally intractible given a large enough n, and also, it has 4 solutions as previously discussed.  Since it has 4 solutions, it's generally unsuitable for the public part, but good for the private part.  If someone uses it for the public part, which one of the 4 solutions should they use?  (There are some work around schemes but they never caught on)

So, the best use of Rabin is for digital signatures.  To digitally sign, we encrypt a message by taking the 4 square roots in modulo n, and select 1 as the digital signature.  The verify the signature, all we have to do is square the digital signature in modulo n, which only has 1 solution, and compare it to the original.  

Since Rabin is so similar to RSA with the difference being the exponents, as you can guess, it's subject to the same homomorphic attacks that RSA is.  So we need to add padding, similar to what we did with RSA, but Rabin specifies a collision resistant hash is needed after the padding.

Steps:

We will use e = 1/2 and d = 2.  we will take a square root to encrypt and take a square to decrypt.

n = pq very similar in sizing to RSA, because if we can factor n into p and q, we can break Rabin.

p and q must be chosen such that p mod 4 = 3 and q mod 4 = 3.  This constraint ensures that both p and q will have 1 of 4 quadratic residues in modulo.  This makes it really easy to find squares because so many exist.

We generate a random pad, concatenate the pad with our message, and take a hash.  Most sources recommend a minimum 256 bit hash at present.  Also, the size of n needs to be at least 4 times the size of the hash, so if we use a 256 bit hash, n needs to be at least 1024 bits.

We must now verify if the hash we just calculated is a square. We have a really quick way to check if the hash is a square. If h^((p-1)/2) mod p = 1 and h^((q-1)/2) mod q = 1, then it's a square!

The digital signature formula takes the roots of h in p and the roots of h in q, selects 1 of each, and then uses the Chinese Remainder Theorem to combine them. (I'll let you look at the code below)  The formula looks really complicated, but it actually has very few operations.

Verification of the digital signature is the best part.  Simply square the digital signature in modulo n and compare it to a hash of the original message and with the padding.  If they match the digital signature is valid.  



In [1]:
import hashlib
from sage.all import *

In [2]:
def my_print_number(label, x):
    "prints a number in decimal, number of digits, hex, number of bits"
    
    print ("\n", label, '\n')
    print ("decimal:", "{:,}".format(x), "\n")
    print ("number of digits:", x.ndigits(), "\n")
    print ("hex:", x.hex(), "\n")
    print ("number of bits:", x.nbits(), "\n")

In [3]:
def my_find_n_p_q(b):
    "find two primes p and q, that satisfy p (mod 4) = 3 and q (mod 4) = 3, that when multiplied together yield n of the given number of bits"
    
    max_tries = 10
    
    b_half = b // 2
    
    upper_limit = (2^b_half) - 1
    lower_limit = (2^(b_half-1))
    
    p = random_prime(upper_limit, false, lower_limit)
    
    tries = 1
    
    while power_mod(p, 1, 4) != 3:
        p = random_prime(upper_limit, false, lower_limit)
        tries += 1
        if tries > max_tries:
            print ("max tries ", max_tries, "could not find a p where p (mod 4) = 3!")
            return
        
    print ("\nfinding p where p (mod 4) = 3 took ", tries, "tries\n")
    
    q = random_prime(upper_limit, false, lower_limit)
    
    tries = 1
    
    while power_mod(q, 1, 4) != 3:
        q = random_prime(upper_limit, false, lower_limit)
        tries += 1
        if tries > max_tries:
            print ("max tries ", max_tries, "could not find a q where q (mod 4) = 3!")
            return
    
    print ("\nfinding q where q (mod 4) = 3 took ", tries, "tries\n")
    
    n = p * q
    
    my_print_number("p", p)
    my_print_number("q", q)
    my_print_number("n", n)
    
    return (p, q, n)

In [4]:
# come back here and re-run with the remaining bit sizes

# since the hash is 256 bits, and there 4 quadratic residues, we must use a minimum bit size of 1024

#(p, q, n) = my_find_n_p_q(1024)
#(p, q, n) = my_find_n_p_q(2048)
(p, q, n) = my_find_n_p_q(4096)



finding p where p (mod 4) = 3 took  1 tries




finding q where q (mod 4) = 3 took  1 tries


 p 

decimal: 21,874,941,132,649,201,033,883,029,325,890,384,000,995,668,447,472,104,501,206,359,188,144,285,604,722,870,455,576,770,755,972,685,486,660,625,942,890,220,459,490,285,027,658,285,714,443,928,101,285,095,141,382,217,525,779,769,937,582,306,979,701,372,330,527,661,099,229,296,708,974,890,354,134,155,680,860,781,524,942,029,702,454,487,377,463,791,543,075,107,458,778,060,005,114,161,285,066,941,109,130,297,937,760,620,918,711,081,030,475,725,498,258,018,378,699,269,082,525,813,311,085,989,502,556,122,171,907,815,933,948,977,909,990,266,429,580,144,376,346,822,287,456,611,349,981,588,843,710,163,356,598,447,269,499,772,760,979,431,052,183,749,615,901,742,763,964,416,720,077,440,253,369,484,198,172,945,661,901,837,977,706,431,959,876,077,985,320,012,112,055,896,508,464,881,934,388,210,672,671,549,603,593,291,020,031 

number of digits: 617 

hex: ad486cf156f1c570085419bc05c970b18756f2a65fac4d2e2c3cb84dad83de71ba4bcc31fcdaf1673041b

In [5]:
# m is our original message

m = 1234567890

In [6]:
def my_pad_hash_verify_repeat(m, p, q):
    "give a message m, pad it, hash it, verify it's a square in modulo n"
    
    max_tries = 100
    
    found = False
    
    tries = 0
    
    while not found:
        
        tries += 1
        if tries > max_tries:
            print ("max tries ", max_tries, "could not find a padded hashed message that was a square in modulo n!")
            return (0, 0)
        
        # find a random pad of size 5
        random_pad = str(randint(10000,99999))
        
        # hash the original message concatenated with the random pad
        h = Integer("0x" + hashlib.sha256((str(m) + random_pad).encode('utf-8')).hexdigest())
        
        if power_mod(h, (p-1)//2, p) != 1:
            continue
            
        if power_mod(h, (q-1)//2, q) != 1:
            continue
            
        found = true
    
    print ("\ntook ", tries, " tries to find a padded hashed message that was a square in modulo n\n")
    
    return random_pad, h

In [7]:
(pad, h) = my_pad_hash_verify_repeat(m, p, q)

print ("h padded hashed message in hex:", hex(h), "\n")

print ("pad:", str(pad))


took  4  tries to find a padded hashed message that was a square in modulo n

h padded hashed message in hex: 0x9b416d15ecb9c0fcc25ddf84c541ef184a40c33a4cd5fa1db896ec9e0ea76ab 

pad: 70699


In [8]:
def my_create_signature(h, n, p, q):
    "given the padded hash h, n, p, q, create the rabin signature in modulo n"
    
    s1 = power_mod(p, q-2, q) * power_mod(h, (q+1)//4, q)
    
    s2 = power_mod(q, p-2, p) * power_mod(h, (p+1)//4, p)
    
    s = power_mod( (s1 * p) + (s2 * q), 1, n)
    
    return s

In [9]:
signature = my_create_signature(h, n, p, q)

my_print_number("rabin signature", signature)


 rabin signature 

decimal: 467,944,690,835,187,459,757,306,440,848,559,951,750,188,099,755,253,892,742,675,269,368,991,833,578,674,937,002,701,315,153,882,597,979,077,070,153,861,142,800,942,299,758,549,040,643,971,841,874,229,359,297,871,877,011,089,073,364,072,004,062,996,700,382,842,364,946,977,910,456,410,087,044,383,267,341,893,707,924,780,407,463,121,650,207,523,409,333,220,188,109,626,704,570,301,245,098,686,070,528,502,660,141,411,735,157,832,156,754,535,917,336,860,752,602,328,615,639,992,917,695,342,164,827,252,199,744,937,195,344,660,557,240,758,716,138,301,089,116,919,718,654,978,299,762,244,554,819,692,236,683,824,060,869,134,680,986,932,304,486,960,437,643,620,011,127,080,665,782,634,672,259,657,066,468,353,713,067,172,157,622,909,322,917,537,317,896,182,040,164,900,660,069,103,270,590,424,939,625,146,210,863,942,803,227,749,565,609,844,321,879,706,998,404,348,830,243,273,086,222,173,064,374,766,268,729,531,596,063,356,639,628,782,418,822,520,521,737,763,372,172,239,082

In [10]:
def my_verify_signature(signature, m, pad, n):
    "given the signature, the original message, the pad used, square the signature in modulo n and verify it matches the hash of the message and pad"
    
    square_of_signature = power_mod(signature, 2, n)
    
    # hash the original message concatenated with the random pad
    hash_message_pad = Integer("0x" + hashlib.sha256((str(m) + pad).encode('utf-8')).hexdigest())
    
    if square_of_signature == hash_message_pad:
        print ("signature is valid!")
    else:
        print ("signature is NOT valid!")
        
    

In [11]:
my_verify_signature(signature, m, pad, n)

signature is valid!
